In [8]:
import os
os.environ["JAVA_HOME"] = "/Users/debasis/devtools/jdk-11.0.10.jdk/Contents/Home/"
print(os.getenv("JAVA_HOME"))

/Users/debasis/devtools/jdk-11.0.10.jdk/Contents/Home/


In [9]:
import pyterrier as pt
from transformers import AutoTokenizer
from collections import Counter
from pyterrier.measures import *

In [10]:
def printTopFrequentTerms(index_ref):
    # Access the lexicon
    lexicon = index_ref.getLexicon()
    
    # Extract terms and their frequencies
    term_freqs = []
    iterator = lexicon.iterator()
    while iterator.hasNext():
        entry = iterator.next()
        term_freqs.append((entry.getKey(), entry.getValue().getFrequency()))
    
    # Sort by frequency and get the top 50 terms
    top_terms = sorted(term_freqs, key=lambda x: x[1], reverse=True)[:50]
    
    # Print results
    for term, freq in top_terms:
        print(f"{term}: {freq}")

In [11]:
import pandas as pd

dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019')

WORD_INDEX_DIR="./wordindex"
word_index_ref = pt.IndexFactory.of(WORD_INDEX_DIR)

print(word_index_ref.getCollectionStatistics().toString())

tfidf = pt.terrier.Retriever(word_index_ref, wmodel="TF_IDF", num_results=100)
#tfidf_gpt = pt.terrier.Retriever(index_ref, wmodel="TF_IDF", properties={"termpipelines" : ""}, num_results=100)  # No tokenization

rm3 = tfidf >> pt.rewrite.RM3(word_index_ref) >> tfidf
pt.Experiment([tfidf, rm3], dataset.get_topics(), dataset.get_qrels(), eval_metrics=[nDCG@10, AP(rel=2), AP(rel=3)],names=["tfidf", "rm3"])

10:32:21.001 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 0
Number of tokens: 288759529
Field names: []
Positions:   false



,name,nDCG@10,AP(rel=2),AP(rel=3)
0,tfidf,0.478310,0.232189,0.163141
1,rm3,0.525136,0.258130,0.194894


In [12]:
# Load Queries from File
query_file = "pass_2019.gpt-tok.queries"  # Change to your actual query file
queries = pd.read_csv(query_file, sep='\t', names=["qid", "query"], dtype={"qid": str, "query": str})

# Load the Indexed Collection
index_dir = "./gpt_index"  # Path to your indexed data
index_ref = pt.IndexFactory.of(index_dir)

print(index_ref.getCollectionStatistics().toString())

tfidf_gpt = pt.terrier.Retriever(index_ref, wmodel="TF_IDF", num_results=100)  # No tokenization
rm3_gpttok_50_20 = tfidf_gpt >> pt.rewrite.stash_results(clear=False) >> pt.rewrite.RM3(index_ref, fb_terms=50, fb_docs=20) >> pt.rewrite.reset_results() >> tfidf_gpt
rm3_gpttok_10_10 = tfidf_gpt >> pt.rewrite.RM3(index_ref, fb_terms=10, fb_docs=10) >> tfidf_gpt
rm3_gpttok_20_20 = tfidf_gpt >> pt.rewrite.RM3(index_ref, fb_terms=20, fb_docs=20) >> tfidf_gpt

pt.Experiment([tfidf_gpt, rm3_gpttok_50_20, rm3_gpttok_10_10, rm3_gpttok_20_20],
              queries, dataset.get_qrels(), eval_metrics=[nDCG@10, AP(rel=2), AP(rel=3)],
              names=["tfidf-gpt-tokens", "tfidf-gpt-tokens-rm3-50,20",
                     "tfidf-gpt-tokens-rm3-10,10", "tfidf-gpt-tokens-rm3-20,20"])

10:33:19.883 [main] WARN org.terrier.structures.BaseCompressingMetaIndex -- Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 2 GiB of memory would be required.
Number of documents: 8841823
Number of terms: 22887
Number of postings: 242159486
Number of fields: 0
Number of tokens: 327156239
Field names: []
Positions:   false



,name,nDCG@10,AP(rel=2),AP(rel=3)
0,tfidf-gpt-tokens,0.421618,0.187597,0.148047
1,"tfidf-gpt-tokens-rm3-50,20",0.471735,0.204556,0.169798
2,"tfidf-gpt-tokens-rm3-10,10",0.473803,0.210617,0.165043
3,"tfidf-gpt-tokens-rm3-20,20",0.471898,0.208558,0.171299
